# Suspicious RUN Key from Download
Detects the suspicious RUN keys created by software located in Download or temporary Outlook/Internet Explorer directories

## Rule Content
```
- title: Suspicious RUN Key from Download
  id: 9c5037d1-c568-49b3-88c7-9846a5bdc2be
  status: experimental
  description: Detects the suspicious RUN keys created by software located in Download
    or temporary Outlook/Internet Explorer directories
  references:
  - https://app.any.run/tasks/c5bef5b7-f484-4c43-9cf3-d5c5c7839def/
  author: Florian Roth
  date: 2019/10/01
  tags:
  - attack.persistence
  - attack.t1060
  logsource:
    product: windows
    service: sysmon
    category: null
  detection:
    selection:
      EventID: 13
      Image:
      - '*\Downloads\\*'
      - '*\Temporary Internet Files\Content.Outlook\\*'
      - '*\Local Settings\Temporary Internet Files\\*'
      TargetObject: '*\SOFTWARE\Microsoft\Windows\CurrentVersion\Run\\*'
    condition: selection
  falsepositives:
  - Software installers downloaded and used by users
  level: high

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-endpoint-winevent-sysmon-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(event_id:"13" AND process_path.keyword:(*\\Downloads\\* OR *\\Temporary\ Internet\ Files\\Content.Outlook\\* OR *\\Local\ Settings\\Temporary\ Internet\ Files\\*) AND registry_key_path.keyword:*\\SOFTWARE\\Microsoft\\Windows\\CurrentVersion\\Run\\*)')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()